<a href="https://colab.research.google.com/github/HuyenNguyenHelen/Huyen_INFO5731_Spring2020/blob/master/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [2019 Dell labtop](https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/dp/B07N49F51N/ref=sr_1_11?crid=1IJ7UWF2F4GHH&keywords=dell%2Bxps%2B15&qid=1580173569&sprefix=dell%2Caps%2C181&sr=8-11&th=1) on amazon.

(2) Collect the top 100 User Reviews of the film [Joker](https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv) from IMDB.

(3) Collect the abstracts of the top 100 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(4) Collect the top 100 tweets by using hashtag ["#wuhancoronovirus"](https://twitter.com/hashtag/wuhancoronovirus) from Twitter. 


**# The top 100 User Reviews of the film Joker from IMDB.**

In [0]:
# Question 1:  Write a python program to collect text data and save the data into a csv file:

# Import essential packages

import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
# Request page to get content
from bs4 import BeautifulSoup
import requests

session = requests.Session()
host_url = 'https://www.imdb.com/title/tt7286456/reviews'
get_webpage = session.get(host_url)
res_host = BeautifulSoup(get_webpage.content, 'html.parser')

# Find key of load-more pages
key = res_host.find('div', {'class':'load-more-data'})
datakey = key.attrs['data-key']

# Generate urls by found keys 
# Request to continue to get next-page key
loadmore_url_list = []
a = True
while a:
    if key is None:
        print('error')
        a = False
    else:
        loadmore_url = 'https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey='+datakey
        get_loadmore = session.get(loadmore_url)
        res_loadmore = BeautifulSoup(get_loadmore.content, 'html.parser')
        key = res_loadmore.find('div', {'class':'load-more-data'})
        datakey = key.attrs['data-key']
        loadmore_url_list.append('https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey='+datakey)
        if len(loadmore_url_list) ==4: # need 4 urls to find around 100 reviews, then stop
          break
print(loadmore_url_list)

['https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cjmquzd63qg7cvh7obzr3q42brhzfmxvlnomwklyczuf43o6ss6oe4ftmjndf4k4e2iyi2r32qpfom2r6dz3oydpoi', 'https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cbkqqzdeyye7owx7ojur7rmwazhzfmxvlnomwklyczuf43o6ss6oe3v3njldv4k5a7iddizbi5rnocjkwfjappkiwa', 'https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cbpry2d6zab7wthhnbzrxumqbzhzfmxvlnomwklyczuf43o6ss6oe3vznzmdv4k57vdyrh2dt7o6d26zplqltokipi', 'https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cboqi2tayyg66uxfojurtu4mcbhzfmxvlnomwklyczuf43o6ss6oe3vjprjcv4k5unuy3mc5li6ujwg4iivt4vcmpa']


In [18]:
# Create a list of all urls, including urls in host and next pages, which will be using
host_url = 'https://www.imdb.com/title/tt7286456/reviews'
all_urls=[]
all_urls.append(host_url)
all_urls.extend(loadmore_url_list)
#user_names, user_links, content, date, title, review_helpfulness = [],[], [], [], [], []
review_elements = []
for url in all_urls:
  request = urllib.request.urlopen (url)
  webpage  = request.read()
  soup = BeautifulSoup(webpage)

  reviews = soup.find_all("div", {"class":"lister-item-content" })
  for review in reviews:
    try:
        review_title = review.find("a", class_="title").text.strip()
    except:
        review_title = ""
    try:
        review_id = review.find("a")["href"].split("/")[2]
    except:
        review_id = ""
    try:
        review_date = review.find("span", class_="review-date").get_text()
    except:
        review_date = ""
    try:
        review_nickname = review.find ("span", class_="display-name-link").get_text()
    except:
        review_nickname = ""
    try:
        review_content = review.find ("div", class_="text show-more__control").get_text()
    except:
        review_content =""
    #print(review_content)
    try:
        review_rating = review. find ("span", class_="rating-other-user-rating").getText().strip()
        #print(review_rating)
    except:
        review_rating = ""
    try:
        review_helpfulness = review.find("div", {"class": "actions text-muted"}).text.split("\n")[1]
        #print(review_helpfulness)
    except:
        review_helpfulness = ""

    review_elements.append([review_title, review_id, review_date, review_nickname, review_content, review_rating, review_helpfulness])
#print(len(review_elements))

df = pd.DataFrame(review_elements, columns=["review_title", "review_id", "review_date", "review_nickname", "review_content", "review_rating", "review_helpfulness"])
df

with open("E:\Helen\INFO5731_Assignment2\Joke_reviews.csv", "w", newline="", encoding='utf-8') as file:
  df.to_csv(file)
with open("E:\Helen\INFO5731_Assignment2\Joke_reviews.csv", "r", encoding='utf-8') as file:
  pd.read_csv(file)



,review_title,review_id,review_date,review_nickname,review_content,review_rating,review_helpfulness
0,As a viewer that actually went to TIFF and wit...,rw5112402,10 September 2019,JF500,I was a person that saw all the hype and claim...,10/10,"6,848 out of 7,870 found t..."
1,Outstanding movie with a haunting performance ...,rw5159304,3 October 2019,MihaVrhunc,"Every once in a while a movie comes, that trul...",10/10,"3,383 out of 4,047 found t..."
2,Only certain people can relate,rw5168360,7 October 2019,lesterarnoldpinto,This is a movie that only those who have felt ...,10/10,"2,687 out of 3,317 found t..."
3,Must have put a SMILE of Satisfaction on Heath...,rw5092831,1 September 2019,Chandler_Bing_,"Truly a masterpiece, The Best film of 2019, on...",10/10,"3,171 out of 3,940 found t..."
4,The Hype is real,rw5160204,4 October 2019,kdagoulis26,Most of the time movies are anticipated like t...,10/10,"2,357 out of 2,923 found t..."
...,...,...,...,...,...,...,...
89,Speechless,rw5161350,4 October 2019,aldoheda,How can i speak something that my feeling cant...,10/10,76 out of 151 found this h...
90,Disturbingly brilliant film making,rw5144817,26 September 2019,ryanphelantv,"Read all you like about ""Joker"" - there is not...",10/10,88 out of 177 found this h...
91,Mental illnesses...,rw5185889,13 October 2019,alics-70920,Do not lead to torturing children (mother) nor...,2/10,54 out of 104 found this h...
92,Overrated,rw5181517,12 October 2019,thebradenz,"I don't get it. I think it's a case of ""everyo...",2/10,54 out of 104 found this h...


# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [19]:
# Write your code here
#clean the text data
from __future__ import division
import nltk # re, pprint
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string 

with open("E:\Helen\INFO5731_Assignment2\Joke_reviews.csv", "r", encoding='utf-8') as file:
  data = pd.read_csv(file)

raw_title = [str(line) for line in data.review_title]
raw_content = [str(line) for line in data.review_content]
# Tokenize raw text in raw_title and raw_content
tokens_title= [nltk.word_tokenize(title) for title in raw_title]
tokens_content = [nltk.word_tokenize(item) for item in raw_content]

# PROCCESSES OF CLEANING TEXT  
def cleaning_text (text):
    step1 = remove_noise(text)
    step2 = remove_num (step1)
    step3 = remove_stop (step2)
    step4 = lowercase (step3)
    step5 = stemming (step4)
    step6 = lemma (step5)
    return step6

# (1) Remove noise, such as special characters and punctuations.      
def remove_noise (list):
    punct=[char for char in string.punctuation]
    noise_removed = [word for word in list if word not in punct]
    return noise_removed

#(2) Remove numbers.
def remove_num (list):
    num_removed  = [word for word in list if not word.isnumeric()]
    return num_removed

 # (3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).
def remove_stop (list):
    stopword_removed = [word for word in list if word not in stopwords.words('english')]
    return stopword_removed

#(4) Lowercase all texts
def lowercase (list):
    lowercase = [word.lower() for word in list]
    return lowercase
# (5) Stemming
def stemming (list):
    ps = PorterStemmer()
    word_stemmed = [ps.stem(word) for word in list]
    return word_stemmed
# (6) Lemmatization.
def lemma (list):
    wnl =  WordNetLemmatizer ()
    word_lemmatized = [wnl.lemmatize(word) for word in list]
    return word_lemmatized

# Cleaning title reviews and content review
cleaned_title = [cleaning_text (sublist) for sublist in tokens_title]
cleaned_content = [cleaning_text (sublist) for sublist in tokens_content]
# Save cleaned text into new columns in dataframe
df["cleaned_title"] = cleaned_title
df["cleaned_content"] = cleaned_content
df






[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


,review_title,review_id,review_date,review_nickname,review_content,review_rating,review_helpfulness,cleaned_title,cleaned_content
0,As a viewer that actually went to TIFF and wit...,rw5112402,10 September 2019,JF500,I was a person that saw all the hype and claim...,10/10,"6,848 out of 7,870 found t...","[a, viewer, actual, went, tiff, wit, film, n't...","[i, person, saw, hype, claim, masterpiec, over..."
1,Outstanding movie with a haunting performance ...,rw5159304,3 October 2019,MihaVrhunc,"Every once in a while a movie comes, that trul...",10/10,"3,383 out of 4,047 found t...","[outstand, movi, haunt, perform, best, charact...","[everi, movi, come, truli, make, impact, joaqu..."
2,Only certain people can relate,rw5168360,7 October 2019,lesterarnoldpinto,This is a movie that only those who have felt ...,10/10,"2,687 out of 3,317 found t...","[onli, certain, peopl, relat]","[thi, movi, felt, alon, isol, truli, relat, yo..."
3,Must have put a SMILE of Satisfaction on Heath...,rw5092831,1 September 2019,Chandler_Bing_,"Truly a masterpiece, The Best film of 2019, on...",10/10,"3,171 out of 3,940 found t...","[must, put, smile, satisfact, heath, ledger, '...","[truli, masterpiec, the, best, film, one, best..."
4,The Hype is real,rw5160204,4 October 2019,kdagoulis26,Most of the time movies are anticipated like t...,10/10,"2,357 out of 2,923 found t...","[the, hype, real]","[most, time, movi, anticip, like, end, fall, s..."
...,...,...,...,...,...,...,...,...,...
89,Speechless,rw5161350,4 October 2019,aldoheda,How can i speak something that my feeling cant...,10/10,76 out of 151 found this h...,[speechless],"[how, speak, someth, feel, cant, even, contain..."
90,Disturbingly brilliant film making,rw5144817,26 September 2019,ryanphelantv,"Read all you like about ""Joker"" - there is not...",10/10,88 out of 177 found this h...,"[disturbingli, brilliant, film, make]","[read, like, ``, joker, '', noth, prepar, gut-..."
91,Mental illnesses...,rw5185889,13 October 2019,alics-70920,Do not lead to torturing children (mother) nor...,2/10,54 out of 104 found this h...,"[mental, ill, ...]","[do, lead, tortur, child, mother, murder, peopl]"
92,Overrated,rw5181517,12 October 2019,thebradenz,"I don't get it. I think it's a case of ""everyo...",2/10,54 out of 104 found this h...,[overr],"[i, n't, get, i, think, 's, case, ``, everyon,..."


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [21]:
# Write your code here
#(1) Tag parts of Speech (POS) and  calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb)...
nltk.download('averaged_perceptron_tagger')
from collections import Counter
count_POStag = []
for tokens in cleaned_title:
    title_tagged = nltk.pos_tag(tokens)
    count_POStag.append(Counter (tag for tokens, tag in title_tagged))
for tokens in cleaned_content:
    content_tagged = nltk.pos_tag(tokens)
    count_POStag. append (Counter (tag for tokens, tag in content_tagged))
    
print(count_POStag)
count_VB=0
count_NN=0
count_ADJ=0
count_ADV=0
for pack in count_POStag:
  for item in pack:
    if item =="VB" or item=="VBG" or item=="VBD" or item=="VBP" or item=="VBZ":
      count_VB+=1
    elif item =="NN" or item=="NNS" or item =="NNP" or item=="NNPS":
      count_NN+=1
    elif item =="JJ" or item =="JJR" or item=="JJS":
      count_ADJ+=1
    elif item =="RB" or item=="RBR" or item=="RBS":
      count_ADV+=1
    else:
      pass
print("The total verbs:", count_VB)
print("The total nouns:", count_NN)
print("The total adjectives:", count_ADJ)
print("The total adverbs:", count_ADV)

  




[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[Counter({'NN': 9, 'JJ': 3, 'DT': 1, 'VBD': 1, 'RB': 1, 'VB': 1}), Counter({'NN': 5, 'JJS': 1, 'VB': 1, 'RB': 1, 'VBN': 1}), Counter({'NN': 3, 'JJ': 1}), Counter({'NN': 4, 'RB': 2, 'MD': 1, 'VB': 1, 'JJ': 1, 'POS': 1}), Counter({'DT': 1, 'NN': 1, 'JJ': 1}), Counter({'NN': 2}), Counter({'RB': 1, 'VB': 1, 'WRB': 1, 'JJ': 1, 'VBP': 1}), Counter({'NN': 2, 'VBD': 1, 'JJ': 1}), Counter({'NN': 3, 'DT': 1, 'RB': 1, 'VBD': 1}), Counter({'NN': 6, 'POS': 1, 'JJS': 1, 'VBZ': 1, ':': 1, 'VBD': 1}), Counter({'JJ': 1, 'NN': 1}), Counter({'JJ': 2, 'NN': 1}), Counter({'JJ': 1, 'NN': 1}), Counter({'NN': 2}), Counter({'NN': 4, 'RB': 2, 'VB': 1}), Counter({'JJ': 1, 'NN': 1}), Counter({'NN': 2}), Counter({'VB': 2, 'VBP': 1, 'RB': 1}), Counter({'NN': 2, 'MD': 1, 'VB': 1}), Counter({'NN': 2}), Counter({'NN': 3, 'CD': 1, 'JJ

In [0]:
# (2)Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences.
# Download and import necessarry packages
import spacy
import spacy.cli
spacy.cli.download("en_core_web_lg")
nlp = spacy.load('en_core_web_lg')
!pip install benepar
import benepar
from nltk.tree import Tree 
benepar.download('benepar_en2')
nltk.download('punkt')


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


[nltk_data] Downloading package benepar_en2 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
# Print out the constituency parsing trees
# Preprocessing for constituency parsing and dependency parsing
def cleaning_sentence (text):
    step1 = remove_noise(text)
    step2 = remove_num (step1)
    step3 = lowercase (step2)
    return step3
text_4parsing = [sent.split(".") for sent in cleaning_sentence(raw_content)]

#print out the constituency parsing 
parser = benepar.Parser("benepar_en2")
for sublist in text_4parsing[1:5]:
  for sents in sublist:
    tree = parser.parse(sents)
    print(tree)
    tree.pretty_print()
 

(S
  (NP (DT every) (RB once))
  (PP (IN in) (NP (DT a) (NN while)))
  (NP (DT a) (NN movie))
  (VP (VBZ comes))
  (, ,)
  (NP (WDT that))
  (ADVP (RB truly))
  (VP (VBZ makes) (NP (DT an) (NN impact))))
                                           S                                            
        ___________________________________|_______________________________              
       |            PP                |          |    |   |     |          VP           
       |         ___|___              |          |    |   |     |      ____|___          
       NP       |       NP            NP         VP   |   NP   ADVP   |        NP       
   ____|___     |    ___|____      ___|____      |    |   |     |     |     ___|____     
  DT       RB   IN  DT       NN   DT       NN   VBZ   ,  WDT    RB   VBZ   DT       NN  
  |        |    |   |        |    |        |     |    |   |     |     |    |        |    
every     once  in  a      while  a      movie comes  ,  that truly makes  an   

ValueError: ignored

In [0]:
# Print out dependence parsing tree
from spacy import displacy
for sublist in text_4parsing[:4]:
    doc = [nlp(sents) for sents in sublist]
for item in doc:
  for token in item:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
displacy.render(doc, style = "dep", jupyter=True, options={"distance":140})



truly truly ADV RB advmod xxxx True False
a a DET DT det x True True
masterpiece masterpiece NOUN NN ROOT xxxx True False
, , PUNCT , punct , False False
the the DET DT det xxx True True
best good ADJ JJS amod xxxx True False
film film NOUN NN appos xxxx True False
of of ADP IN prep xx True True
2019 2019 NUM CD pobj dddd False False
, , PUNCT , punct , False False
one one NUM CD appos xxx True True
of of ADP IN prep xx True True
the the DET DT det xxx True True
best good ADJ JJS amod xxxx True False
films film NOUN NNS pobj xxxx True False
of of ADP IN prep xx True True
the the DET DT det xxx True True
decade decade NOUN NN pobj xxxx True False
    SPACE _SP    False False
and and CCONJ CC cc xxx True True
truly truly ADV RB advmod xxxx True False
the the DET DT det xxx True True
best good ADJ JJS amod xxxx True False
film film NOUN NN ROOT xxxx True False
to to PART TO aux xx True True
bring bring VERB VB acl xxxx True False
a a DET DT det x True True
comic comic ADJ JJ amod xxxx Tru

In [0]:
#(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.
hist = {}
for sublist in text_4parsing:
    doc = [nlp(sents) for sents in sublist]
    for item in doc:
      item.ents
      for ent in item.ents:
        print(ent.text, ent.label_)
        hist[ent.label_]=hist.get(ent.label_,0)+1
print(hist)

a day DATE
2019 DATE
one CARDINAL
the decade DATE
first ORDINAL
yesterday DATE
2019 DATE
5 CARDINAL
this year DATE
10 CARDINAL
100% PERCENT
years ago DATE
1 MONEY
the decade DATE
dozens CARDINAL
one CARDINAL
three CARDINAL
90% PERCENT
ten CARDINAL
year DATE
10/10 CARDINAL
2 hour TIME
2 hours TIME
one CARDINAL
today DATE
first ORDINAL
80 CARDINAL
8 CARDINAL
today DATE
first ORDINAL
80 CARDINAL
8 CARDINAL
10/10 CARDINAL
the year DATE
10 CARDINAL
the year nor DATE
the year DATE
recent years DATE
one CARDINAL
& ORG
10/10 CARDINAL
2 CARDINAL
2 hours TIME
3 and a half CARDINAL
5 minutes TIME
these couple hours TIME
second ORDINAL
10 CARDINAL
first ORDINAL
the last decade DATE
half CARDINAL
$$$ MONEY
10/10 CARDINAL
one CARDINAL
two CARDINAL
1 CARDINAL
2 CARDINAL
3 CARDINAL
half CARDINAL
still days DATE
the night TIME
31 CARDINAL
76th ORDINAL
the entire year DATE
1970s DATE
two CARDINAL
first ORDINAL
second ORDINAL
half CARDINAL
third ORDINAL
one CARDINAL
these few years DATE
100% PERCENT
one 

**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

I pulled out one sentence from my data to compare the differences between the constituency parsing tree and dependency parsing tree: 
"i was a person that saw all the hype and claims of masterpiece as overreacting and overblown excitement for another joker based film"
The constituency parsing tree breaks the sentence into two main elements: Subject and Verb Phrase. Then, they are continously splited up into smaller units. In particular, Verb Phrase is broken into Verb and Noun Phrase; next, this is divided into Verb Phrase and Subordinate Clause, etc. The processes of splitting elements into smaller units are implemented from left to right until they can not be broken any more. 
In contrast, having a look the displayed dependency parsing tree, the process is pretty different. It works on each single element and its relationship with other element. What's more, it not only examines relationships of each element with the one in its right, but also to its left. For example, the verb "was" has two-directed relationship with subject "I" and "person".



In [0]:
#print out the constituency parsing 
sample = [sent for sent in text_4parsing[0]]
print (sample[0])
parser = benepar.Parser("benepar_en2")
tree = parser.parse(sample[0])
print(tree)
tree.pretty_print()

# Print out dependence parsing tree
doc = nlp(sample[0])
displacy.render(doc, style = "dep", jupyter=True, options={"distance":140})


i was a person that saw all the hype and claims of masterpiece as overreacting and overblown excitement for another joker based film
(S
  (NP (PRP i))
  (VP
    (VBD was)
    (NP
      (NP (DT a) (NN person))
      (SBAR
        (WHNP (WDT that))
        (S
          (VP
            (VBD saw)
            (NP
              (NP (PDT all) (DT the) (NN hype) (CC and) (NNS claims))
              (PP (IN of) (NP (NNP masterpiece))))
            (PP
              (IN as)
              (NP
                (VP (VBG overreacting))
                (CC and)
                (JJ overblown)
                (NN excitement)
                (PP
                  (IN for)
                  (NP
                    (DT another)
                    (ADJP (NN joker) (VBN based))
                    (NN film)))))))))))
     S                                                                                                                                                 
  ___|____________                       